In [7]:
from acquire import *
from prepare import *
import unicodedata
import re
import json
import os
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from nltk.corpus import stopwords
from time import strftime

import scipy.stats as stats
from jos_acquire import *

import warnings
warnings.filterwarnings("ignore")

## Acquire
___

In [8]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        git = 'https://github.com/'
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links

In [11]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

repository_links

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


['/apple/llvm-project',
 '/apple/foundationdb',
 '/apple/swift',
 '/apple/swift-docc-render',
 '/apple/swift-corelibs-libdispatch',
 '/apple/swift-package-manager',
 '/apple/swift-experimental-string-processing',
 '/apple/swift-protobuf',
 '/apple/swift-syntax',
 '/apple/sourcekit-lsp',
 '/apple/swift-argument-parser',
 '/apple/swift-docc',
 '/apple/swift-llbuild',
 '/apple/swift-source-compat-suite',
 '/apple/swift-markdown',
 '/apple/swift-docc-symbolkit',
 '/apple/swift-docc-render-artifact',
 '/apple/swift-lmdb',
 '/apple/swift-tools-support-core',
 '/apple/swift-driver',
 '/apple/indexstore-db',
 '/apple/swift-stress-tester',
 '/apple/swift-xcode-playground-support',
 '/apple/swift-integration-tests',
 '/apple/swift-cmark',
 '/apple/swift-corelibs-foundation',
 '/apple/swift-corelibs-xctest',
 '/apple/swift-docker',
 '/apple/swift-installer-scripts',
 '/apple/swift-llbuild2',
 '/apple/swift-evolution',
 '/apple/servicetalk',
 '/apple/swift-nio-ssh',
 '/apple/password-manager-resou

In [22]:
df = pd.read_json('data.json')

df

,repo,language,readme_contents
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...
3,apple/llvm-project,None,# Apple's fork of llvm-project\n\nThis is Appl...
4,apple/swift,C++,"<img src=""https://swift.org/assets/images/swif..."
...,...,...,...
149,apple/ccs-pyopendirectory,C++,
150,apple/swift-protobuf-plugin,None,"<img src=""https://swift.org/assets/images/swif..."
151,apple/ccs-caldavclientlibrary,HTML,README for CalDAVClientLibrary\n\nINTRODUCTION...
152,apple/ccs-pyosxframeworks,Python,Getting Started\n===============\n\nThis is a ...


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             154 non-null    object
 1   language         146 non-null    object
 2   readme_contents  154 non-null    object
dtypes: object(3)
memory usage: 3.7+ KB


In [14]:
df.language.value_counts()

Swift               65
Python              38
C++                 15
C                   12
JavaScript           3
HTML                 2
Shell                2
Jupyter Notebook     2
Java                 2
LLVM                 1
Dockerfile           1
R                    1
Starlark             1
Markdown             1
Name: language, dtype: int64

Our categories for classification will most likely be Swift, Python, C (with both C & C++), and other. We can make a new column mapping that target.

In [20]:
df[df.isnull()]

,repo,language,readme_contents
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
149,NaN,NaN,NaN
150,NaN,NaN,NaN
151,NaN,NaN,NaN
152,NaN,NaN,NaN


In [21]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 153
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             146 non-null    object
 1   language         146 non-null    object
 2   readme_contents  146 non-null    object
dtypes: object(3)
memory usage: 4.6+ KB
